In [9]:
import pandas as pd
import numpy as np
import sklearn as sk
import tensorflow as tf
import matplotlib.pyplot as plt
import nibabel as nib
import os
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

#This block for playing with Excel Data
df = pd.read_excel("PTs_500_4k_blinded.xlsx")
df.head()
print(df.info)

del_indices = set() #To keep in track with the deleted values
# Filter out rows with negative values
#This is means that we do have (171-163+1 negative values)
negative_values_filter = (df.iloc[:, 1] < 0) | (df.iloc[:, 2] < 0)
df_filtered = df[~negative_values_filter]
del_indices.update(df[negative_values_filter].index)

#getting red of the outliers
#This is means that we do have 1 outlier
Z_scores = stats.zscore(df_filtered.iloc[:,1])
threshold = 3
outliers_filter = Z_scores >= threshold
df_without_outliers = df_filtered[~outliers_filter]
del_indices.update(df_filtered[outliers_filter].index)
print(df_without_outliers.info)

#Our final preprocessed data is df_without_outliers
# To keep track of the deleted indices, to delete the associated photos as well
print(del_indices)

dir = r'/Users/mathewbernard/downloads/8650dataset'

nii_data = []

for file in os.listdir(dir):
    if file.endswith('.nii'):
        filepath = os.path.join(dir, file)
        img = nib.load(filepath)
        data = img.get_fdata()
        nii_data.append(data)
        #plt.imshow(data[:, :, data.shape[2] // 2], cmap='gray')  # Display a slice from the middle of the image
        #plt.title(f'Sample MRI Image - {file}')
        #plt.colorbar()
        #plt.show()
        
#print(len(nii_data))
nii_data_array = np.array(nii_data)

#Normalize the image data to be between Zero and 1
#before normalization
print(f'Max Before Normalization: {nii_data_array.max()}')
#After Normalization
nii_data_reshaped = nii_data_array.reshape(-1,nii_data_array.shape[-1])
scaler = MinMaxScaler()
nii_data_normalized = scaler.fit_transform(nii_data_reshaped)
nii_data_normalized = nii_data_normalized.reshape(nii_data_array.shape)
print(f'Max After Normalization: {nii_data_normalized.max()}')

X = nii_data_normalized # Image Data
y = df_without_outliers[['PT500','PT4000']].values #Threeshold Data

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)
X_train, X_eval, y_train,y_eval = train_test_split(X,y,test_size=0.1,random_state=42)

print("Completed run one")


<bound method DataFrame.info of              ID  PT500  PT4000
0    smwp1_0001   25.0    72.5
1    smwp1_0002    5.0    10.0
2    smwp1_0003    5.0    45.0
3    smwp1_0004    5.0    15.0
4    smwp1_0005    2.5    62.5
..          ...    ...     ...
166  smwp1_0167    0.0    27.5
167  smwp1_0168   37.5     2.5
168  smwp1_0169    7.5    10.0
169  smwp1_0170    0.0    -2.5
170  smwp1_0171   10.0    57.5

[171 rows x 3 columns]>
<bound method DataFrame.info of              ID  PT500  PT4000
0    smwp1_0001   25.0    72.5
1    smwp1_0002    5.0    10.0
2    smwp1_0003    5.0    45.0
3    smwp1_0004    5.0    15.0
4    smwp1_0005    2.5    62.5
..          ...    ...     ...
165  smwp1_0166    2.5     2.5
166  smwp1_0167    0.0    27.5
167  smwp1_0168   37.5     2.5
168  smwp1_0169    7.5    10.0
170  smwp1_0171   10.0    57.5

[162 rows x 3 columns]>
{104, 169, 51, 19, 85, 118, 23, 57, 26}
Max Before Normalization: 1.1022199392318726
Max After Normalization: 1.0
Completed run one


In [18]:
from tensorflow.keras import regularizers
print("running build model")

# Reshape data
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)
X_val_reshaped = X_eval.reshape(X_eval.shape[0], X_eval.shape[1], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)

print(X_test_reshaped.size)
print(y_test.size)

#model definiton - regularizers apply L2
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=64, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.SimpleRNN(units=64, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.SimpleRNN(units=64, kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(units=1, kernel_regularizer=regularizers.l2(0.01))
])

# Compile
print("running compilation of model")
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Training
print("training model")
history = model.fit(X_train_reshaped, y_train,
                    validation_data=(X_val_reshaped, y_eval),
                    batch_size=16,
                    epochs=25)

print("Shape of X_test_reshaped:", X_test_reshaped.shape)
print("Shape of y_test:", y_test.shape)

# Evaluate the model
print("evaluating model")
loss = model.evaluate(X_test_reshaped, y_test_subset)
mae = model.evaluate(X_test_reshaped, y_test_subset)
print("Test Loss:", loss)
print("Test MAE:", mae)

print("completed model build and evaluation")




running build model
57728649
66
running compilation of model
training model
Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 225ms/step - loss: 762.0740 - mae: 20.9046 - val_loss: 789.9417 - val_mae: 17.9922
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 626.3597 - mae: 17.1944 - val_loss: 705.9169 - val_mae: 17.1599
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - loss: 560.8741 - mae: 16.3753 - val_loss: 667.5400 - val_mae: 16.8673
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - loss: 536.2114 - mae: 15.7132 - val_loss: 641.1530 - val_mae: 16.6525
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - loss: 472.5447 - mae: 14.6368 - val_loss: 618.2202 - val_mae: 16.5357
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - loss: 452.2332 - mae: 14.4397 - val_loss: 600.0858 - val_mae: 16.4529
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - loss: 453.2070 - mae: 14.4849 - val_loss: 583.8818 - val_mae: 16.3722
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - lo